# Setup

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from deap import algorithms, base, creator, tools

In [19]:
# Toy data
toy_data = pd.DataFrame(data=[[(0.011, 5),(0.03, 6),(0.05, 7),(0.067, 8),(0.09, 9)],
            [(1, 9),(0.67, 6),(0.33, 3),(0.11, 1),(0.05, 5)],
            [(0.75, 0.5),(2.25, 3),(3.5, 5),(5.3, 8),(6.5, 10)],
            [(5.3, 0.1),(9.1, 0.8),(5.04, 0.05),(6.4, 0.3),(8.02, 0.6)]])

In [42]:
MA_df = pd.DataFrame(columns=['activator','inhibitor'],data=[[0,-2],[0.5,-0.7],[0.7,-0.3],[2,0]])
MA_df

,activator,inhibitor
0,0.0,-2.0
1,0.5,-0.7
2,0.7,-0.3
3,2.0,0.0


# DEAP

In [3]:
creator.create(name = 'fitness',
               base = base.Fitness,
               weights = (-1.0,))

creator.create(name = 'individual',
               base = np.ndarray,
               fitness = creator.fitness)

toolbox = base.Toolbox()

/opt/miniconda3/envs/sym-seq-ml/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/miniconda3/envs/sym-seq-ml/lib/python3.10/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [167]:
random.seed(42)

def create_individual(individual_class: type, data: pd.DataFrame):
    dimensions = data.shape
    locations = random.sample(range(dimensions[1]),dimensions[0])
    individual = individual_class(data.iloc[i,locations[i]] for i in data.index)

    return individual

In [168]:
toolbox.register(alias = 'individual',
                 function = create_individual,
                 individual_class = creator.individual,
                 data = toy_data) # CHANGE

toolbox.register('population',
                 tools.initRepeat,
                 list,
                 toolbox.individual)

In [169]:
# Evaluate
def objective(individual: object, MA_df: pd.DataFrame):
    dot_product = []
    for i in MA_df.index:
        dot_product.append((np.dot([MA_df.iloc[i,0],MA_df.iloc[i,1]],individual[i]))**2)
    sum_dots = np.sum(dot_product)
    
    return -sum_dots,

toolbox.register(alias = 'evaluate', function = objective, MA_df = MA_df)

In [170]:
# NOTE: Not sure what appropriate tournament size should be
toolbox.register("select", tools.selTournament, tournsize=5)

# NOTE: Using np.ndarray individuals requires a different method to compare individuals
hof = tools.HallOfFame(3, similar=np.array_equal)

In [208]:
# Mutation
def mutate(individual: object, prob: float, data: pd.DataFrame):
    dimensions = data.shape
    for i, vals in enumerate(individual):
        if random.random() < prob:
            individual[i] = list(data.iloc[i,random.sample(range(dimensions[1]),1)])[0]

    return individual,

toolbox.register('mutate', function = mutate, prob = 1, data = toy_data) # CHANGE

In [218]:
# Crossover
toolbox.register('mate', tools.cxESBlend, alpha=0.1)

In [219]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [225]:
MU = 5
LAMBDA = 10
pop = toolbox.population(n=30)

pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
            cxpb=0, mutpb=0.3, ngen=100, stats=stats, halloffame=hof, verbose=False)

In [226]:
logbook

[{'gen': 0,
  'nevals': 30,
  'avg': -401.63297083333333,
  'std': 90.76225514651428,
  'min': -547.8240999999999,
  'max': -248.424925},
 {'gen': 1,
  'nevals': 2,
  'avg': -514.4859450000001,
  'std': 18.69611783767153,
  'min': -533.386725,
  'max': -491.72485},
 {'gen': 2,
  'nevals': 2,
  'avg': -527.79665,
  'std': 0.0,
  'min': -527.79665,
  'max': -527.79665},
 {'gen': 3,
  'nevals': 4,
  'avg': -527.79665,
  'std': 0.0,
  'min': -527.79665,
  'max': -527.79665},
 {'gen': 4,
  'nevals': 3,
  'avg': -624.9881600000001,
  'std': 79.35653561020285,
  'min': -689.7824999999999,
  'max': -527.79665},
 {'gen': 5,
  'nevals': 3,
  'avg': -657.38533,
  'std': 64.79433999999996,
  'min': -689.7824999999999,
  'max': -527.79665},
 {'gen': 6,
  'nevals': 1,
  'avg': -657.3853299999998,
  'std': 64.79433999999995,
  'min': -689.7824999999999,
  'max': -527.79665},
 {'gen': 7,
  'nevals': 2,
  'avg': -689.7824999999999,
  'std': 0.0,
  'min': -689.7824999999999,
  'max': -689.7824999999999}

In [227]:
toolbox.evaluate(hof[0])

(-690.5961,)